# Boxing Simulation

## Patrick Maloney
### DATA604 Final Project

## Introduction

Boxing is one of the world's oldest professional sports, but it has been one of the slowest to be inoavted by analytics and data science applications that now proliferate other professional sports. In this project, I will attempt to build a simulation of the biggest match in history, Floyd Mayweather Jr. vs. Manny Pacquiao.  The match was one of the biggest financial events in the history of sports, netting each fighter nine-figure paydays for an evening's work, grossing over \$600 million overall, and leading to a then-record of over \\$50 million dollars being wagered on the fights outcome.  I will attempt to use punch statistics from both fighters to simulate the fight with Python.


### Import libraries

In [1]:
import numpy
import pandas
import random
from modsim import *
%matplotlib inline
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
# set the random number generator
np.random.seed(7)

### Build State objects for each fighter

I will be using the modsim library for its convenience in building simulations. I build a state object for each fighter, which will  store each of there metrics from the simulaiton.

In [2]:
mayweather = State(thrown = 0, landed = 0, kd = 0, score = 0)
pacquiao = State(thrown = 0, landed = 0, kd = 0, score = 0)

,values
thrown,0
landed,0
kd,0
score,0


### Time-step function
Now we need to create a time-step function that will simulate an instance in a round, so that we can later loop over to simulate an entire round. In this step function, we will use punch statistics to determine the probabilities that a punch will be thrown during a given moment, and whether that punch will land. 

In [3]:
def step(system):
    """This function takes in a system argument 
    containing punch probabilities and increments
    the state objects for each fighter"""
    if flip(system.m_throw): #probability Mayweather throws a punch
        mayweather.thrown += 1 # increment Mayweather "thrown" stats
        print('may throw')
        if flip(system.m_land): #probability the thrown punch lands on Pacquiao
            mayweather.landed += 1 # increment Mayweather "landed" stats
            print('may land')

In [4]:
# We define a system of parameters that correspond with the probability that each fighter
# will throw and/or land a punch during any given second of a round. these are placeholder values to get started.
system = System(m_throw = 0.21, m_land = 0.48, p_throw = 0.28, p_land = 0.3 )

,values
m_throw,0.21
m_land,0.48
p_throw,0.28
p_land,0.30


In [5]:
step(system)

may throw


In [6]:
mayweather

,values
thrown,1
landed,0
kd,0
score,0


We see the function works as intended for a single fighter. Now let's add the second fighter to the mix, since boxing takes two to tango.

In [7]:
def step(system):
    """This function takes in a system argument 
    containing punch probabilities and increments
    the state objects for each fighter"""
    #Mayweather
    if flip(system.m_throw): #probability Mayweather throws a punch
        mayweather.thrown += 1 # increment Mayweather "thrown" stats
        print('may throw')
        if flip(system.m_land): #probability the thrown punch lands on Pacquiao
            mayweather.landed += 1 # increment Mayweather "landed" stats
            print('may land')
    #Pacquiao
    if flip(system.p_throw): #probability Pacquiao throws a punch
        pacquiao.thrown += 1 # increment Pacquiao "thrown" stats
        print('pac throw')
        if flip(system.p_land): #probability the thrown punch lands on Mayweather
            pacquiao.landed += 1 # increment Pacquiao "landed" stats
            print('pac land')

In [8]:
step(system)

In [9]:
mayweather

,values
thrown,1
landed,0
kd,0
score,0


In [10]:
pacquiao

,values
thrown,0
landed,0
kd,0
score,0


It worked as intended after I ran the cell multiple times. The next thing I will need is a function that will initialize the state objects back to zero to make testing these functions easier.

In [11]:
def reset_stats(state):
    "Takes state object as argument and resets punch counts"
    state.thrown = 0
    state.landed = 0
    state.kd = 0
    state.score = 0

reset_stats(mayweather)
mayweather

,values
thrown,0
landed,0
kd,0
score,0


In [12]:
reset_stats(pacquiao)
pacquiao

,values
thrown,0
landed,0
kd,0
score,0


### Simulating a round
Now lets run the time-step function in a loop 180 times to simulate a 3-minute round. At the end of the round, we will want to compare the fighters landed punches and award more points to the person who landed more punches. Boxing uses a scoring system that awards 10 points to the winner and 9 points to the loser (minus a point for a knockdown, but we'll deal with that later.)  I will also remove the print statements from the step function.

In [13]:
def step(system):
    """This function takes in a system argument 
    containing punch probabilities and increments
    the state objects for each fighter"""
    #Mayweather
    if flip(system.m_throw): #probability Mayweather throws a punch
        mayweather.thrown += 1 # increment Mayweather "thrown" stats
        if flip(system.m_land): #probability the thrown punch lands on Pacquiao
            mayweather.landed += 1 # increment Mayweather "landed" stats
  
    #Pacquiao
    if flip(system.p_throw): #probability Pacquiao throws a punch
        pacquiao.thrown += 1 # increment Pacquiao "thrown" stats
        if flip(system.p_land): #probability the thrown punch lands on Mayweather
            pacquiao.landed += 1 # increment Pacquiao "landed" stats

            
def sim_round(system):
    for i in range(180): # run step function for each second in 3 minute round
        step(system)
    if mayweather.landed > pacquiao.landed:
        mayweather.score += 10
        pacquiao.score += 9
    elif pacquiao.landed > mayweather.landed:
        mayweather.score += 9
        pacquiao.score += 10
    else:
        mayweather.score += 10
        pacquiao.score += 10
        

sim_round(system)
mayweather

,values
thrown,42
landed,17
kd,0
score,10


In [14]:
pacquiao

,values
thrown,52
landed,15
kd,0
score,9


The round_sim function appears to work as intended. However, for it to work in a loop to simulate an entire fight, we will need  the counts to reset at the beginning of each round after the points have been awarded, but still increment the totals for each fighter. To do this, I will create a new state object within the system object that will store the values for that round and increment the totals for each fighter. I will also include a new function to reset the values at the beginning of each round.

In [25]:
init = State(m_thrown = 0,
             m_landed = 0,
             m_kd = 0,
             p_thrown = 0,
             p_landed = 0,
             p_kd = 0)

system = System(init = init,
                m_throw = 0.21,
                m_land = 0.48,
                p_throw = 0.28,
                p_land = 0.30)

def reset_round_stats():
    system.init = State(m_thrown = 0,
             m_landed = 0,
             m_kd = 0,
             p_thrown = 0,
             p_landed = 0,
             p_kd = 0)

def step(system):
    """This function takes in a system argument 
    containing punch probabilities and increments
    the state objects for each fighter"""
    #Mayweather
    if flip(system.m_throw): #probability Mayweather throws a punch
        system.init.m_thrown += 1 # increment Mayweather round "thrown" stats
        if flip(system.m_land): #probability the thrown punch lands on Pacquiao
            system.init.m_landed += 1 # increment Mayweather "landed" stats for this round
  
    #Pacquiao
    if flip(system.p_throw): #probability Pacquiao throws a punch
        system.init.p_thrown += 1 # increment Pacquiao "thrown" stats
        if flip(system.p_land): #probability the thrown punch lands on Mayweather
            system.init.p_landed += 1 # increment Pacquiao "landed" stats

            
def sim_round(system):
    """Takes in a system object that includes punch counters for the round.
    Passes the system argument through the step function in a loop.
    increments state object parameters based on loop results."""
    
    reset_round_stats() # clear punch stats of previous round
    for i in range(180): # run step function for each second in 3 minute round
        step(system)
    if system.init.m_landed > system.init.p_landed:
        mayweather.score += 10
        pacquiao.score += 9
    elif system.init.p_landed > system.init.m_landed:
        mayweather.score += 9
        pacquiao.score += 10
    else:
        mayweather.score += 10
        pacquiao.score += 10
    # transfer punch stats for this round to cumulative totals
    mayweather.thrown += system.init.m_thrown 
    mayweather.landed += system.init.m_landed
    mayweather.kd += system.init.m_kd
    pacquiao.thrown += system.init.p_thrown
    pacquiao.landed += system.init.p_landed
    pacquiao.kd += system.init.p_kd

In [16]:
reset_stats(mayweather)
reset_stats(pacquiao)
sim_round(system)

In [17]:
mayweather

,values
thrown,46
landed,20
kd,0
score,10


In [18]:
pacquiao

,values
thrown,54
landed,16
kd,0
score,9


### Simulating a fight
Now that a round can be simulated, we can simulate a whole fight by looping the sim_round function over the scheduled number of rounds.

In [19]:
def sim_fight(system, n_rounds):
    """takes in a system and an integer and simulates that number of rounds"""
    # create scorecards
    mayweather_scorecard = []
    pacquiao_scorecard = []
    Round = []
    may_throw = []
    may_land = []
    pac_throw = []
    pac_land = []
    # clear punch stats
    reset_stats(mayweather)
    reset_stats(pacquiao)
    # sim number of rounds
    for n in linspace(1,n_rounds, n_rounds):
        sim_round(system)
        Round.append(int(n))
        may_throw.append(system.init.m_thrown)
        may_land.append(system.init.m_landed)
        pac_throw.append(system.init.p_thrown)
        pac_land.append(system.init.p_landed)
        mayweather_scorecard.append(mayweather.score)
        pacquiao_scorecard.append(pacquiao.score)
    d = {'Mayweather': mayweather_scorecard,
         'Pacquiao': pacquiao_scorecard,
         'M_Thrown': may_throw,
         'M_Landed': may_land,
         'P_Thrown': pac_throw,
         'P_Landed': pac_land}
    scorecard = DataFrame(data=d, index= Round)
    return scorecard

sim_fight(system, 12)

,Mayweather,Pacquiao,M_Thrown,M_Landed,P_Thrown,P_Landed
1,9,10,37,20,60,22
2,19,19,40,19,43,16
3,29,28,42,19,55,13
4,39,37,35,23,51,13
5,48,47,24,11,55,13
6,58,56,37,15,44,11
7,68,65,34,19,64,18
8,78,74,34,21,43,16
9,88,83,46,23,62,19
10,98,92,42,17,50,15


In [20]:
mayweather

,values
thrown,451
landed,227
kd,0
score,118


In [21]:
pacquiao

,values
thrown,616
landed,189
kd,0
score,111


### Knockdowns and KOs
The fight simulation seems to work as it should. We are able to take in a system of fight parameters and simulate an entire fight and detrmine the winner by who wins the most rounds, which is in turn based on who lands the most punches during that round.  It's exciting to see it working, but much work remains.  We are simulating a boxing match, not a fencing match. In boxing, a fighter can be knocked down or knocked out by any given punch. We will need to rework our step function to reflect this aspect of the fight.

In [199]:
def step(system):
    """This function takes in a system argument 
    containing punch probabilities and increments
    the state objects for each fighter"""
    #Mayweather
    if flip(system.m_throw): #probability Mayweather throws a punch
        system.init.m_thrown += 1 # increment Mayweather round "thrown" stats
        if flip(system.m_land): #probability the thrown punch lands on Pacquiao
            system.init.m_landed += 1 # increment Mayweather "landed" stats for this round
            if flip(system.m_kd): #probability landed punch causes a knockdown
                system.init.m_kd += 1 # increment knockdown counter
                if flip(system.m_ko):   # probability that knocked down opponent is knocked out
                    return "Mayweather by K.O."
                
  
    #Pacquiao
    if flip(system.p_throw): #probability Pacquiao throws a punch
        system.init.p_thrown += 1 # increment Pacquiao "thrown" stats
        if flip(system.p_land): #probability the thrown punch lands on Mayweather
            system.init.p_landed += 1 # increment Pacquiao "landed" stats
            if flip(system.p_kd): #probability landed punch causes a knockdown
                system.init.p_kd += 1 # increment knockdown counter
                if flip(system.p_ko):   # probability that knocked down opponent is knocked out
                    return "Pacquiao by K.O."
            
system = System(init = init,  # these are again placeholder values to test the functions
                m_throw = 0.21,
                m_land = 0.48,
                m_kd = 0.5,  # add probability that a landed punch will produce a knockdown
                m_ko = 0.30,  # add probability that a knocked down opponent is knocked out
                p_throw = 0.28,
                p_land = 0.30,
                p_kd = 0.5,   
                p_ko = 0.45)

,values
init,m_thrown 32 m_landed 16 m_kd 0 p...
m_throw,0.21
m_land,0.48
m_kd,0.5
m_ko,0.3
p_throw,0.28
p_land,0.3
p_kd,0.5
p_ko,0.45


In [314]:
# testing the new function
step(system)
init

,values
m_thrown,50
m_landed,28
m_kd,7
p_thrown,77
p_landed,21
p_kd,3


After running the above cell multiple times with the exaggerated parameters, it seems to work as intended. However, the return statement in the event of a KO will abruptly end a simulation if it ocurrs, and will not provide us with a scorecard to let us know what round it occurred in and what happened up to that point. I will try to create a function that deploys a scorecard in the event of a KO.


In [420]:
def ko(fighter):
    '''Takes a string with the name of the victorious fighter.
    Returns a statement of the KO and a scorecard of the fight to that point'''
    print("{fighter} wins by K.O.".format(fighter = fighter))

 

def step(system):
    """This function takes in a system argument 
    containing punch probabilities and increments
    the state objects for each fighter"""
    #Mayweather
    if flip(system.m_throw): #probability Mayweather throws a punch
        system.init.m_thrown += 1 # increment Mayweather round "thrown" stats
        mayweather.thrown += 1 # increment cumulative stats
        if flip(system.m_land): #probability the thrown punch lands on Pacquiao
            system.init.m_landed += 1 # increment Mayweather "landed" stats for this round
            mayweather.landed += 1 # increment cumulative stats
            if flip(system.m_kd): #probability landed punch causes a knockdown
                system.init.m_kd += 1 # increment knockdown counter
                mayweather.kd += 1  # increment cumulative stats
                if flip(system.m_ko):   # probability that knocked down opponent is knocked out
                    ko("Mayweather")
                    return True
                
  
    #Pacquiao
    if flip(system.p_throw): #probability Pacquiao throws a punch
        system.init.p_thrown += 1 # increment Pacquiao "thrown" stats
        pacquiao.thrown += 1  # increment cumulative stats
        if flip(system.p_land): #probability the thrown punch lands on Mayweather
            system.init.p_landed += 1 # increment Pacquiao "landed" stats
            pacquiao.landed += 1  # increment cumulative stats
            if flip(system.p_kd): #probability landed punch causes a knockdown
                system.init.p_kd += 1 # increment knockdown counter
                pacquiao.kd += 1  # increment cumulative stats
                if flip(system.p_ko):   # probability that knocked down opponent is knocked out
                    ko("Pacquiao")
                    return True
                
            
def sim_round(system):
    """Takes in a system object that includes punch counters for the round.
    Passes the system argument through the step function in a loop.
    increments state object parameters based on loop results."""
    
    reset_round_stats() # clear punch stats of previous round
    for i in range(180): # run step function for each second in 3 minute round
        res = step(system)
        if res == True:
              return res
              break
    if system.init.m_landed > system.init.p_landed or system.init.m_kd > system.init.p_kd: #add logic for winning round via KD
        mayweather.score += 10
        pacquiao.score += 9 - system.init.m_kd  #subtract a point for each KD
    elif system.init.p_landed > system.init.m_landed or system.init.m_kd < system.init.p_kd:
        mayweather.score += 9 - system.init.p_kd
        pacquiao.score += 10
    else:
        mayweather.score += 10
        pacquiao.score += 10
   
        
mayweather_scorecard = []
pacquiao_scorecard = []
Round = []
may_throw = []
may_land = []
may_kd = []
pac_throw = []
pac_land = []
pac_kd = []

def sim_fight(system, n_rounds):
    """takes in a system and an integer and simulates that number of rounds"""
    # create scorecards
    mayweather_scorecard = []
    pacquiao_scorecard = []
    Round = []
    may_throw = []
    may_land = []
    may_kd = []
    pac_throw = []
    pac_land = []
    pac_kd = []
    # clear punch stats
    reset_stats(mayweather)
    reset_stats(pacquiao)
    # sim number of rounds
    for n in linspace(1,n_rounds, n_rounds):
        res = sim_round(system)
        Round.append(int(n))
        may_throw.append(system.init.m_thrown)
        may_land.append(system.init.m_landed)
        pac_throw.append(system.init.p_thrown)
        pac_land.append(system.init.p_landed)
        may_kd.append(system.init.m_kd)
        pac_kd.append(system.init.p_kd)
        mayweather_scorecard.append(mayweather.score)
        pacquiao_scorecard.append(pacquiao.score)
        if res == True:
            break
    d = {'Mayweather': mayweather_scorecard,
         'Pacquiao': pacquiao_scorecard,
         'M_Thrown': may_throw,
         'M_Landed': may_land,
         'M_KD': may_kd,
         'P_Thrown': pac_throw,
         'P_Landed': pac_land,
         'P_KD': pac_kd}
    scorecard = DataFrame(data=d, index= Round)
    return scorecard

sim_fight(system, 12)

,Mayweather,Pacquiao,M_Thrown,M_Landed,M_KD,P_Thrown,P_Landed,P_KD
1,10,9,43,21,0,53,13,0
2,19,19,33,13,0,49,18,0
3,29,28,31,16,0,55,7,0
4,38,38,32,14,0,60,20,0
5,48,47,35,18,0,55,17,0
6,58,56,43,22,0,54,18,0
7,68,65,35,21,0,47,16,0
8,78,74,34,15,0,45,14,1
9,88,83,46,19,0,52,13,0
10,98,92,45,24,0,43,12,0


In [422]:
# let's test with more realistic kd and ko parameters
system = System(init = init,  # these are again placeholder values to test the functions
                m_throw = 0.21,
                m_land = 0.48,
                m_kd = 0.01,  # add probability that a landed punch will produce a knockdown
                m_ko = 0.30,  # add probability that a knocked down opponent is knocked out
                p_throw = 0.28,
                p_land = 0.30,
                p_kd = 0.01,   
                p_ko = 0.35)

sim_fight(system, 12)

Pacquiao wins by K.O.


,Mayweather,Pacquiao,M_Thrown,M_Landed,M_KD,P_Thrown,P_Landed,P_KD
1,9,10,32,17,0,62,23,0
2,19,19,38,19,0,48,16,0
3,29,28,33,22,0,41,14,0
4,39,37,43,20,0,54,14,0
5,49,45,42,22,1,48,12,0
6,58,55,40,14,0,48,17,0
7,68,65,34,16,0,50,16,0
8,78,74,32,16,0,41,8,0
9,88,83,43,19,0,50,15,0
10,88,83,11,7,0,18,8,1


In [417]:
sim_fight(system, 12)

Pacquiao wins by K.O.


,Mayweather,Pacquiao,M_Thrown,M_Landed,M_KD,P_Thrown,P_Landed,P_KD
1,10,9,43,18,0,50,12,0
2,20,19,32,15,0,43,15,0
3,30,27,37,16,1,53,18,0
4,39,37,27,9,0,42,12,0
5,49,46,36,17,0,45,13,0
6,59,55,38,18,0,46,16,0
7,69,64,38,18,0,52,15,0
8,78,74,33,12,0,65,25,0
9,88,82,47,22,1,46,13,0
10,98,91,37,18,0,38,13,1


In [418]:
pacquiao

,values
thrown,552
landed,172
kd,2
score,100


In [419]:
mayweather

,values
thrown,417
landed,187
kd,2
score,108
